In [12]:
import clr
from bonsai_pylib import load_bonsai_config
load_bonsai_config()

# Add reference to the Harp package
clr.AddReference("Harp.VestibularH1")
import Harp.VestibularH1 as h1Dev
clr.AddReference("Bonsai.Harp")
import Bonsai.Harp as harp


In [13]:
## Connect to the device usign the Async Device class
dev = h1Dev.VestibularH1Device("COM5")
# Call the Dispose method to close the connection
#dev.Dispose()

UnauthorizedAccessException: Access to the port 'COM5' is denied.
   at System.IO.Ports.InternalResources.WinIOError(Int32 errorCode, String str)
   at System.IO.Ports.SerialStream..ctor(String portName, Int32 baudRate, Parity parity, Int32 dataBits, StopBits stopBits, Int32 readTimeout, Int32 writeTimeout, Handshake handshake, Boolean dtrEnable, Boolean rtsEnable, Boolean discardNull, Byte parityReplace)
   at System.IO.Ports.SerialPort.Open()
   at Bonsai.Harp.AsyncDevice..ctor(String portName)
   at Harp.VestibularH1.VestibularH1Device..ctor(String portName)

In [ ]:
# Get the map of available registers
RegisterMap = h1Dev.Device().RegisterMap
AvailableKeys = list(h1Dev.Device().RegisterMap.Keys)

# Get the Name of the register:
print(RegisterMap[AvailableKeys[0]])

Cam0Event


In [ ]:
for key in AvailableKeys:
    method = f"Read{RegisterMap[key]}Async"
    try:
        result = getattr(dev, method)().Result
        print(result)
    except:
        pass

0
0
100
0
0
68
0
Harp.VestibularH1.InStatePayload
0
0
0
7


In [ ]:
# Make an asynchronous call to read a register from the device
Cam0Freq = dev.ReadCam0TriggerFrequencyAsync().Result
print([type(Cam0Freq), Cam0Freq])

# Some registers returned complex parsed values that can be accessed through properties
InState = dev.ReadInStateAsync().Result
print([type(InState), InState])
print(InState.Lick0)
print(InState.Lick0Changed)
#...

[<class 'int'>, 100]
[<class 'Harp.VestibularH1.InStatePayload'>, <Harp.VestibularH1.InStatePayload object at 0x000001BCD8091C40>]
False
False


In [ ]:
# Return an harp message from an async read
# Note, the correct method still needs to be applied (i.e. "ReadByte")
harpMessage = dev.CommandAsync(harp.HarpCommand.ReadByte(h1Dev.InState.Address)).Result
print(harpMessage)

Read 39 TimestampedU8@8391.2975 Length:1


In [ ]:
## Write to a register
value = 20
Cam0Freq = dev.WriteCam0TriggerFrequencyAsync(value).Result
# Confirm the value of the register has changed by reading it back
Cam0Freq = dev.ReadCam0TriggerFrequencyAsync().Result
print(f"Write value: {value}, Read value: {Cam0Freq}")

Write value: 20, Read value: 20


In [ ]:
# Write an harp message directly
# Note, the correct method still needs to be applied (i.e. "WriteUInt16")
harpMessage = dev.CommandAsync(
    harp.HarpCommand.WriteUInt16(
        h1Dev.Cam0TriggerFrequency.Address, 100)).Result
print(harpMessage)

Write 34 TimestampedU16@8391.7495 Length:1


In [ ]:
# Manipulate HarpMessage
an_harp_message = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(an_harp_message))
print(harp.HarpMessage.get_PayloadType(an_harp_message))
print(harp.HarpMessage.GetPayloadSingle(an_harp_message))

32
Float
1.0


In [ ]:
h1Dev.Dispose() # Make sure to dispose the device when done


AttributeError: Dispose

# Notes
- Make the AsyncDevice constructor have a generic name instead of using the device name...

- How to apply a "parse" or "format" on an HarpMessage? Should Cam0TriggerFrequency.Format(value) do this?

- Is there a way to write it a register without knowing its type? # E.g.:
```python
command = h1Dev.Cam0TriggerFrequencyCommand()
command.Value = 200

harpMessage = dev.CommandAsync(
    harp.HarpCommand.Write(command)).Result
print(harpMessage)
```